<font color="red"> Hi! My name is Mikhail and I'm your reviewer for this project. You will find my comments in red </font>

<p><font color="blue"> Hi Mikhail. I'm Mike.  Thank you for reviewing my submission! </font></p>

<font color="red"> Hi Mike! Thank you for taking my comments into account and for the additional analysis. The submission is perfect now </font>

## Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

### Step 1. Open the data file and have a look at the general information. 

In [646]:
import pandas as pd
credit_data = pd.read_csv('/datasets/credit_scoring_eng.csv')
#check the general info of the data frame
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Conclusion

When reading the general information of the credit_data table, I observed the following:

1. There are 12 columns with 21,525 rows of data.
2. There are missing values with 'days_employed' and 'total_income', both are float data types.  These values are most likely MNAR since the exact same amount is missing from both columns.




### Step 2. Data preprocessing

### Processing missing values

In [647]:
#identify the missing values
credit_data.isnull().sum()

#fill the cells that have 'NaN' with '0'
credit_data['days_employed'] = credit_data['days_employed'].fillna(0)
credit_data['total_income'] = credit_data['total_income'].fillna(0) 

#check to see if the code worked
credit_data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Conclusion

1. There are 2,174 values missing in the 'days_employed' and 'total_income' columns.  The values in both columns are float data types.  

2. There are many possible reasons these values are missing.  
    1. the customer did not enter the information.
    2. there was a glitch in an automated system.
    3. the customer is unemployed.
    4. the customer is dependent on a spouse for income.
    5. the customer is retired and living off savings.
    6. the customer is living off savings regardless of any other factor.
    7. the customer just started a business and hasn't become profitable.
    8. the customer is recently divorced, leaving the customer without access to the spouse's funds.
    9. the customer is recently widow/widower, leaving the customer without access to the spouse's funds.

3. I used the .fillna() method to fill in the missing values.  This method identifies every cell that contains 'NaN' as a value, and replaces it with a '0'.  


<font color="red"> Good! </font>

<font color="blue"> Thanks! </font>



### Data type replacement

In [648]:
#converts float type to int type
credit_data['days_employed'] = credit_data['days_employed'].astype('int')
credit_data['total_income'] = credit_data['total_income'].astype('int')

#check the column for unique values
credit_data['education'].value_counts()

#convert all values to lowercase letters
credit_data['education'] = credit_data['education'].str.lower()

#check the column again, no duplicates are found
print(credit_data['education'].value_counts())

#check the table to make sure the conversion worked
#print(credit_data.head(20))


secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64


<font color="red"> Why are you reading the data again? I see that you are doing it each time. There is no need to do it. All variables are saved between your cells. So you can read data once and use each cell for some additonal preproessing steps on your data. Just reuse the same variables names. See example with three cells before  </font>  

<font color="blue"> I wasn't sure how Jupyter Notebook worked.  I learned that I need to "Run All" cells in order for my code to work.  Now that I know how to do that, I deleted the repeated code.  



In [649]:
import pandas as pd
credit_data = pd.read_csv('/datasets/credit_scoring_eng.csv')

In [650]:
credit_data['days_employed'] = credit_data['days_employed'].fillna(0)
credit_data['total_income'] = credit_data['total_income'].fillna(0) 

In [651]:
credit_data['days_employed'] = credit_data['days_employed'].astype('int')
credit_data['total_income'] = credit_data['total_income'].astype('int')

<font color="red"> ... further steps with no need to read data and repeat all these steps again and again ... </font>

### Conclusion

I converted the values in columns 'days_employed' and 'total_income' using the .astype() method.  This method takes an exisiting data type and converts it into another data type, in this case 'float' to 'int'.  Converting to integers makes the data easier to digest.  Now that every value is the int type, arithmetic operations can be carried out with the values.



### Processing duplicates

In [652]:
#check for duplicates in the data table
print(credit_data.duplicated().sum())

#drop duplicate data using the .drop_duplicats() method
credit_data = credit_data.drop_duplicates().reset_index()

#check the table again
print(credit_data.duplicated().sum())


54
0


<font color="red"> Great, but these changes won't be applied once you read your data again :( </font>

<font color="blue"> Should be fixed now! </font>

### Conclusion

#### I used the .duplicated() and .sum() method in tandem to find how many duplicates are in the data.  I then deleted the 54 duplicates using the .drop_duplicates() method.  The possible reasons for duplicate data are:
1. Importing data from multiple sources
2. Customer data being transferred from old systems
3. The data is not set up to alert duplicates


<font color="red"> Valid reasons indeed </font>

### Categorizing Data

In [653]:
#categorize total income based on parameters
def class_group(row):

    income = row['total_income']
    
    if income < 20000:
        return 'lower'
    if 20000 <= income < 40000:
        return 'lower middle'
    if 40000 <= income < 70000:
        return 'middle'
    if 70000 <= income < 100000:
        return 'upper middle'
    if income >= 100000:
        return 'upper'

#create a new column that categorizes income by class
credit_data['class_group'] = credit_data.apply(class_group, axis=1)
#print(credit_data)

#check the amount of values that fit into each category
print(credit_data['class_group'].value_counts())
print()

#EDITS BELOW

#lemmatize the column 'purpose'
def purpose_lemma(row):
    import nltk
    from nltk.stem import WordNetLemmatizer
    wordnet_lemma = WordNetLemmatizer()
    lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in nltk.word_tokenize(row)]
    return lemmas

credit_data['purpose'] = credit_data['purpose'].apply(purpose_lemma)

#convert lemmatized values to a broader category
def purpose_group(row):
    purpose = row['purpose']
    
    if 'wedding' in purpose:
        return 'wedding'
    
    if 'car' in purpose:
        return 'car'
    
    if 'property' in purpose or 'real' in purpose or 'house' in purpose or 'housing' in purpose:
        return 'real estate'
    
    if 'university' in purpose or 'education' in purpose or 'educated' in purpose:
        return 'education'

credit_data['purpose'] = credit_data.apply(purpose_group, axis=1)
    
#check the lemmatized values in order to find the common words

print(credit_data['purpose'].value_counts())




lower           9489
lower middle    9170
middle          2434
upper middle     279
upper             99
Name: class_group, dtype: int64

real estate    10814
car             4308
education       4014
wedding         2335
Name: purpose, dtype: int64


### Conclusion

Data categorization is used to get clearer insights by organizing the selected data into groups according to specific criteria.  In this case I categorized the 'total_income' column into 5 classes: lower, lower middle, middle, upper middle, and upper.  By categorizing this data, I can now refer to 5 distinct classes of income when drawing conclusions, rather than specific income values. 

<font color="blue"> *Edit*:
    
I also lemmatized the 'purpose' column and arranged the values into four broad categories: real estate, car, education, and wedding.  By categorizing this data, I can identify certain purposes as being in the same category, even if the input is not worded in exactly the same way. </font>

<font color="red"> Please add purpose categorization. You should use Lemmatizer for that </font>

### Step 3. Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [654]:
#first I want to check the data in the 'children' column
kids_amount = credit_data['children'].value_counts()

#create a log that compares # of kids to defaults
kids_log = credit_data[['children','debt']]

#print table with debt values for each category of kids
kids_grouped = kids_log.groupby('children').sum()

#EDITS BELOW

#create a variable that contains all customers that have kids except for the outlier of 20
has_kids = (kids_log['children'] <= 5) & (kids_log['children'] > 0)

#how many of those customers with kids have debt
has_kids_debt = (kids_log['children'] <= 5) & (kids_log['children'] > 0) & (kids_log['debt'] == 1)

#ratio of defaults for customers woth kids
has_kids_ratio = ((has_kids_debt.sum() / has_kids.sum())*100).round(2)

#find percentage of default based on number of kids
no_kids = (kids_grouped['debt'][0] / kids_amount[0]*100).round(2)
one_kid = (kids_grouped['debt'][1] / kids_amount[1]*100).round(2)
two_kids = (kids_grouped['debt'][2] / kids_amount[2]*100).round(2)
three_kids = (kids_grouped['debt'][3] / kids_amount[3]*100).round(2)
four_kids = (kids_grouped['debt'][4] / kids_amount[4]*100).round(2)
five_kids = (kids_grouped['debt'][5] / kids_amount[5]*100).round(2)

#print percentage results
print('Percentage of past defaults based on number of kids')
print('---------------------------------------------------')
print('0 kids:',no_kids,'%')
print('1 kid:',one_kid,'%')
print('2 kids:',two_kids,'%')
print('3 kids:',three_kids,'%')
print('4 kids:',four_kids,'%')
print('5 kids:',five_kids,'%')
print()
print('Percentage of past defaults based on having any amount of kids')
print('--------------------------------------------------------------')
print('no kids:', no_kids,'%')
print('has kids:', has_kids_ratio,'%')


Percentage of past defaults based on number of kids
---------------------------------------------------
0 kids: 7.54 %
1 kid: 9.23 %
2 kids: 9.45 %
3 kids: 8.18 %
4 kids: 9.76 %
5 kids: 0.0 %

Percentage of past defaults based on having any amount of kids
--------------------------------------------------------------
no kids: 7.54 %
has kids: 9.24 %


### Conclusion

Based on the percentages, I don't believe there is a relationship between having kids and paying back a loan on time.  Having 4 kids gives the customer the highest chance of not paying back the loan, while having 5 kids gives them the lowest chance.  Although the pool of customers with 5 kids is very small, every customer had always paid back their loan on time.

<font color="blue"> *Edit*: 
    
When comparing percentages of customers who have no kids and have any amount of kids, there is a jump in default rates from 7.54 % to 9.24 %.  Therefore having kids does seem to have an effect on whether or not a customer will default.

<font color="red"> Well, we see some difference between 0 kids and having >0 kids. It's just a couple of percents but it's up to our customer to decide whether it's a significant difference or not </font>

- Is there a relation between marital status and repaying a loan on time?

In [655]:
#check the amount of each family status
family_amount = credit_data['family_status'].value_counts()

#create a dictionary for family status
family_dict = credit_data[['family_status_id', 'family_status']]

#drop the duplicates and reset the index
family_dict = family_dict.drop_duplicates().reset_index(drop=True)

#create a category to compare family status to debt
family_log = credit_data[['family_status_id', 'debt']]

#compare amount of defaults per status
family_grouped = family_log.groupby('family_status_id').sum()

#find percentage of default per status
married_final = (((family_grouped['debt'][0] / family_amount[0])*100).round(2))
civil_final = (((family_grouped['debt'][1] / family_amount[1])*100).round(2))
widow_final = (((family_grouped['debt'][2] / family_amount[4])*100).round(2))
divorced_final = (((family_grouped['debt'][3] / family_amount[3])*100).round(2))
unmarried_final = (((family_grouped['debt'][4] / family_amount[2])*100).round(2))

#print percentage results
print('Percentage of past defaults by Marital Status')
print('---------------------------------------------')
print('Married:',married_final,'%')
print('Civil Partnership:',civil_final,'%')
print('Widow / Widower:',widow_final,'%')
print('Divorced:',divorced_final,'%')
print('Unmarried:',unmarried_final,'%')

Percentage of past defaults by Marital Status
---------------------------------------------
Married: 7.54 %
Civil Partnership: 9.32 %
Widow / Widower: 6.57 %
Divorced: 7.11 %
Unmarried: 9.75 %


### Conclusion

I used a dictionary for the family id.  This helped me keep track of the marital status when doing arithmetic operations.  There seems to be a relationship between marital status and repaying a loan on time.  The lowest chance of default belongs to widow/widowers and divorcees, while civil partnerships and unmarried cutomers have the highest risk of defaulting.  

- Is there a relation between income level and repaying a loan on time?

In [656]:
#categorize total income based on parameters
def class_group(row):

    income = row['total_income']
    
    if income < 20000:
        return 'lower'
    if 20000 <= income < 40000:
        return 'lower middle'
    if 40000 <= income < 70000:
        return 'middle'
    if 70000 <= income < 100000:
        return 'upper middle'
    if income >= 100000:
        return 'upper'

#create a new column that categorizes income by class
credit_data['class_group'] = credit_data.apply(class_group, axis=1)

#check the amount of values that fit into each category
class_final = credit_data['class_group'].value_counts()

#create a log based on class
class_log = credit_data[['class_group', 'debt']]

#sum the amount of defaults per class
class_default = class_log.groupby('class_group').sum().sort_values('debt', ascending=False)

#divide class_default by class_final to get percentage of defaults by class
lower_default = ((class_default['debt'][0] / class_final[0])*100).round(2)
lm_default = ((class_default['debt'][1] / class_final[1])*100).round(2)
middle_default = ((class_default['debt'][2] / class_final[2])*100).round(2)
um_default = ((class_default['debt'][3] / class_final[3])*100).round(2)
upper_default = ((class_default['debt'][4] / class_final[4])*100).round(2)

#print the results
print('Percentage of past defaults by class')
print('------------------------------------')
print('Lower:',lower_default,'%')
print('Lower Middle:',lm_default,'%')
print('Middle:',middle_default,'%')
print('Upper Middle:',um_default,'%')
print('Upper:',upper_default,'%')
print()
print()

#create pivot table to examine defaults by class and income type
print('Amount of defaults by class arranged by income types')
print('----------------------------------------------------')
data_pivot = credit_data.pivot_table(index=['income_type'], columns='class_group', values='debt', aggfunc='sum')
#fill the missing value
data_pivot = data_pivot.fillna(0)
print(data_pivot)



Percentage of past defaults by class
------------------------------------
Lower: 8.2 %
Lower Middle: 8.39 %
Middle: 7.07 %
Upper Middle: 5.73 %
Upper: 6.06 %


Amount of defaults by class arranged by income types
----------------------------------------------------
class_group                  lower  lower middle  middle  upper  upper middle
income_type                                                                  
business                     127.0         182.0    57.0    4.0           6.0
civil servant                 41.0          36.0     9.0    0.0           0.0
employee                     484.0         474.0    92.0    2.0           9.0
entrepreneur                   0.0           0.0     0.0    0.0           0.0
paternity / maternity leave    1.0           0.0     0.0    0.0           0.0
retiree                      124.0          77.0    14.0    0.0           1.0
student                        0.0           0.0     0.0    0.0           0.0
unemployed                     1

<font color="red"> Great usage of pivot tables! </font>

### Conclusion

According to the percentage of defaults by class, I believe income has an affect on repaying a loan on time.  It seems the more money the customer makes, the more likely they are to repay their loan on time.  I also used a pivot table to examine if the income type had any effect on the conclusion.  Regardless of income type, higher class customers have less defaults.

- How do different loan purposes affect on-time repayment of the loan?

In [657]:
#check the amount for each purpose
purpose_final= credit_data['purpose'].value_counts()

#create a log based on purpose
purpose_log = credit_data[['purpose', 'debt']]

#group by purpose 
purpose_grouped = purpose_log.groupby('purpose').sum()

#amount of default by each purpose
purpose_default = purpose_grouped.sort_values('debt', ascending=False)

#EDITS BELOW

#percentage of defaults by purpose
real_estate_final = (((purpose_default['debt'][0] / purpose_final[0])*100).round(2))
car_final = (((purpose_default['debt'][1] / purpose_final[1])*100).round(2))
education_final = (((purpose_default['debt'][2] / purpose_final[2])*100).round(2))
wedding_final = (((purpose_default['debt'][3] / purpose_final[3])*100).round(2))

#print the results
print('Percentage of past defaults by purpose')
print('--------------------------------------')
print('Real Estate:',real_estate_final,'%')
print('Car:',car_final,'%')
print('Education:',education_final,'%')
print('Wedding:',wedding_final,'%')




Percentage of past defaults by purpose
--------------------------------------
Real Estate: 7.23 %
Car: 9.35 %
Education: 9.22 %
Wedding: 7.97 %


<font color="red"> Please use Lemmatizer for purposes categorization. Actually there should be just several groups </font>

### Conclusion

I'm not sure how to breakdown the data from here.  It seems that there is a similar ratio of defaults to purpose, which means there is not a relation of purpose to defaulting.

<font color="blue">*Edit*:

It seems that anything real estate related has the lowest chance of default, while using the money for a car is the highest risk.  This makes sense, since real estate is usually an asset while cars are liabilities.</font>

### Step 4. General conclusion

In conclusion, there were two factors that determined default risk: martial status, and total income.  The lower the class, the more at risk the customer was to defaulting.  A customer also had a high risk of defaulting if they were in a civil partnership or unmarried.  There were many artifacts in th data.  For example, there were customers who had -1 kids.  I did not consider them in the final analysis.  

<font color="blue">*Edit*:
    
In addition, I found two more factors that determined default risk: having kids, and purpose for the loan.  Customers with kids of any amount were more likely to default, and customers who use the loan to purchase a car are also more likely to default.  

<font color="red"> Good job. Please review comments </font>